In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
# Disable TensorFlow Caching
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

# Ensure limited GPU memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Batch sizes for testing
BATCH_SIZES = [1, 4, 8, 16, 32, 64]
NUM_RUNS = 5  # Runs per batch size

# Load NASNetMobile model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow function to avoid extra optimizations
@tf.function(experimental_relax_shapes=True)
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 11:56:05.361585: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 11:56:05.361604: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 11:56:05.361608: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 11:56:05.361777: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 11:56:05.361788: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Create random input tensor
        input_tensor = tf.random.uniform((batch_size, 224, 224, 3), dtype=tf.float32)
        
        with tf.device(device):
            #layer_times = {}
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

            # Manually clear TensorFlow cache after each batch run
            del input_tensor
            tf.keras.backend.clear_session

    return results

In [4]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []
for device in devices:
    for batch_size in BATCH_SIZES:
        try:
            tf.profiler.experimental.start(f"logs/Noise/{device[1:4]}/{batch_size}")
            final_results.extend(benchmark_model(device, batch_size))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 11:56:07.751226: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:56:07.751234: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 11:56:09.422779: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 11:56:10.451604: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:56:10.522378: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:56:10.522922: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/1/plugins/profile/2025_02_05_11_56_10/C17586.xplane.pb
2025-02-05 11:56:10.530939: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:56:10.530947: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 4


2025-02-05 11:56:13.588103: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:56:13.665771: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:56:13.665965: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/4/plugins/profile/2025_02_05_11_56_13/C17586.xplane.pb
2025-02-05 11:56:13.676861: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:56:13.676870: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


...

2025-02-05 11:57:18.248329: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:57:18.261017: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:57:18.261150: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/16/plugins/profile/2025_02_05_11_57_18/C17586.xplane.pb
2025-02-05 11:57:18.262449: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:57:18.262452: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 32


2025-02-05 11:57:25.263806: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:57:25.278436: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:57:25.278660: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/32/plugins/profile/2025_02_05_11_57_25/C17586.xplane.pb
2025-02-05 11:57:25.280457: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:57:25.280465: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 64


2025-02-05 11:57:51.541239: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:57:51.581466: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:57:51.582293: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/64/plugins/profile/2025_02_05_11_57_51/C17586.xplane.pb


In [5]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.041951
1,/CPU:0,1,0.037546
2,/CPU:0,1,0.048674
3,/CPU:0,1,0.036619
4,/CPU:0,1,0.042082


In [6]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                              
                         min      mean       var       max
device batch_size                                         
/CPU:0 1            0.036619  0.041374  0.000023  0.048674
       4            0.069200  0.078259  0.000199  0.103204
       8            0.095292  0.108372  0.000130  0.125739
       16           0.178033  0.210936  0.000530  0.231310
       32           0.526176  0.579448  0.007234  0.725690
       64           1.264451  1.359067  0.009104  1.499878
/GPU:0 1            0.087786  0.093507  0.000045  0.104888
       4            0.092173  0.098472  0.000069  0.112974
       8            0.090664  0.095134  0.000034  0.104857
       16           0.090950  0.097586  0.000079  0.110094
       32           0.144339  0.711024  1.528005  2.922150
       64           1.127303  1.225656  0.008355  1.366470